In [1]:
from IPython.display import clear_output

In [2]:
import random
import json
from pprint import pprint

import torch
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

from transformers import AutoModel

import numpy as np

c:\Users\Telha Bilal\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from ibm_watsonx_ai.foundation_models import Model

In [4]:
test_df = pd.read_csv('train_data.csv')

In [5]:
author_to_idx_mapping = {name: i for i, name in enumerate(test_df['AuthorName'].sort_values().unique().tolist())}
idx_to_author_mapping = {v: k for k, v in author_to_idx_mapping.items()}

## Initializing the model

In [6]:
class TextClassifier(nn.Module):
    def __init__(self, input_size=1024, hidden_size=512, num_classes=10):
        super(TextClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.ln1 = nn.LayerNorm(hidden_size)  # LayerNorm after the first layer
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size // 2)
        self.ln2 = nn.LayerNorm(hidden_size // 2)  # LayerNorm after the second layer
        self.fc3 = nn.Linear(hidden_size // 2, num_classes)

    def forward(self, x):
        x = self.fc1(x)
        x = self.ln1(x)  # Apply layer normalization
        x = self.relu(x)
        x = self.fc2(x)
        x = self.ln2(x)  # Apply layer normalization
        x = self.relu(x)
        x = self.fc3(x)
        x = torch.softmax(x, dim=-1)
        return x

In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [8]:
embedding_model = AutoModel.from_pretrained("jinaai/jina-embeddings-v3", trust_remote_code=True).to(device)

clear_output()

In [9]:
input_dim = 1024
hidden_dim = 1024
n_classes = len(author_to_idx_mapping)

classification_head = TextClassifier(input_dim, hidden_dim, n_classes)
classification_head.load_state_dict(torch.load('author_classifier_head_jina_embeddings.pt'))
classification_head.to(device)

TextClassifier(
  (fc1): Linear(in_features=1024, out_features=1024, bias=True)
  (ln1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  (relu): ReLU()
  (fc2): Linear(in_features=1024, out_features=512, bias=True)
  (ln2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  (fc3): Linear(in_features=512, out_features=10, bias=True)
)

In [10]:
def get_classification_logits(inputs):
    
    with torch.no_grad():

        embeddings = embedding_model.encode(inputs, max_length=2048)
        embeddings = torch.from_numpy(embeddings).to(device)
        class_logits = classification_head(embeddings)
    
    return class_logits


def classify(inputs):

    
    logits = get_classification_logits(inputs)
    classes = torch.argmax(logits, -1)
    
    return classes.tolist()


def classify_and_get_prob(inputs, author_names):

    if isinstance(author_names, str):
        author_names = [author_names]*len(inputs)

    logits = get_classification_logits(inputs).tolist()

    probs = [logit[author_to_idx_mapping[author_name]] for logit, author_name in zip(logits, author_names)]
    return probs


In [11]:
tdf = test_df.sample(n=2).copy()
tauthors = tdf['AuthorName'].tolist()
classes = classify(tdf['ChapterText'].tolist())

In [12]:
touts = [idx_to_author_mapping[i] for i in classes]

print(tauthors, touts)

['زكي نجيب محمود', 'نوال السعداوي'] ['زكي نجيب محمود', 'نوال السعداوي']


## Preparing ALLAM API model

In [13]:
credentials = {"url": "https://eu-de.ml.cloud.ibm.com", "apikey": "JsA-9hFRsH1msnDzUkcRpoUhSi3r3dfQMAj8Ic9yRiNg"}

project_id = "43ada3b1-5d11-42e8-9ef1-c718b99c05e1"
model_id = "sdaia/allam-1-13b-instruct"

parameters = {
    # "decoding_method": "greedy",
    "max_new_tokens": 500,
    # "repetition_penalty": 1.05
}

In [14]:
# Initialize the model
model = Model(
    model_id=model_id,
    params=parameters,
    credentials=credentials,
    project_id=project_id
)

## Testing the performance

In [17]:
def preprocess_text(text):

    words = text.split()
    words = words[:500]
    processed_text = ' '.join(words)

    return processed_text

In [18]:
PROMPT_TEMPLATE = """

1. أنت كاتب عربي خبير.
2. سيتم تزويدك بنص إدخال ونص مرجعي.
3. مهمتك هي إعادة صياغة النص المُدخل وإعادة كتابته ليتوافق مع أسلوب كتابة النص المرجعي.
4. يمكنك تغيير المفردات والقواعد وما إلى ذلك لمطابقة نمط كتابة النص الناتج مع النص المرجعي، ولكن لا ينبغي أن يتغير المعنى الفعلي للنص المدخل
5. لا تخرج أي شيء إضافي. فقط النص المعاد كتابته. لا تقم بإخراج وصف النص

<Example_1>
    <مرجعي>
    أيهما أكبر: الكون أم الحياة الإنسانية؟ إن الحياة إن لم تكن لها غاية بعيدة موصولة بالغاية التي يسعى إليها الكون برمته فهي ولا ريب أصغر من أن تقاس إليه، أو يفاضل بينها وبينه. وقد كان يكفينا على هذا الفرض كرتنا الأرضية وحدها أو نظام واحد من أنظمة الشموس التي لا عداد لها. وإذا كانت الحياة الإنسانية هي الحس الشاعر المفرد في الوجود، فلِمَ لم يكن لها من الإحساس القدر الكافي لمعرفة الوجود حق المعرفة؟ ولِمَ لَمْ يتناسب العارف والمعروف أو يتقاربا؟ ألا نفهم من ذلك أنه لا بد في الوجود من قدرة تعرفه المعرفة الخليقة به؟ هذا هو الخاطر الذي قام بنفسي عند نظم الأبيات الآتية
    </مرجعي>

    <إدخال>
    لِمَ قبُح الثناء في الوجه حتى تواطئوا على تزييفه؟ ولِمَ حسُن في المغيب حتى تُمُنِّيَ ذلك بكل معنًى؟ ألأن الثناء في الوجه أشبه الملق والخديعة وفي المغيب أشبه الإخلاص والتَّكْرِمَة أم لغير ذلك؟ قال أبو علي مسكويه، رحمه الله: لما كان الثناء في الوجه على الأكثر إعارة شهادةٍ بفضائل النفس وخديعة الإنسان بهذه الشهادة حتى صار ذلك — لاغتراره وتركه كثيرًا من الاجتهاد في تحصيل الفضائل، وغرض فاعلِ ذلك احتراز مودة صاحبه إلى
    </إدخال>

    <إعادة كتابة المدخلات>
    لِمَ كان الثناء في الحضور مكروهًا حتى أَجْمَعَ الناس على زيفه؟ ولِمَ كان الثناء في الغياب محمودًا حتى تَحَبَّبَتْ إليه النفوس في كل مقام؟ أَلِأَنَّ الثناء في الحضور أقرب إلى المداهنة والخداع، وفي الغياب أقرب إلى الإخلاص والتكريم؟ أم أنَّ في الأمر معنى آخر؟ فقد قال أبو علي مسكويه، رحمه الله: إذ كان الثناء في الحضور في الغالب شهادة مُعارَة على فضائل النفس، وخديعةً للنفس بهذه الشهادة، حتى صار ذلك سببًا لانخداع المرء وتركه السعي الجاد لتحصيل الفضائل، وغرض المادح بذلك إرضاء صديقه والتودد إليه.
    </إعادة كتابة المدخلات>
</Example_1>

<Example_2>
    <مرجعي>
    {reference_text}
    </مرجعي>

    <إدخال>
    {input_text}
    </إدخال>

    <إعادة كتابة المدخلات>

"""

In [19]:
n_exp_per_author = 3
# author_to_rewrites = {}
rewrites_data = []

for author in tqdm(author_to_idx_mapping.keys()):

    author_mask = test_df['AuthorName']==author
    author_df = test_df[author_mask]
    non_author_df = test_df[~author_mask]

    prompts = []
    input_samples = []

    for _ in range(n_exp_per_author):

        author_example = author_df.sample(n=1)['ChapterText'].apply(preprocess_text).iloc[0]
        input_row = non_author_df.sample(n=1).iloc[0]

        input_author_name = input_row['AuthorName']
        input_text = preprocess_text(input_row['ChapterText'])

        prompt = PROMPT_TEMPLATE.format(reference_text=author_example,
                                        input_text=input_text)
        
        input_samples.append([author, author_example, input_author_name, input_text])
        prompts.append(prompt)
        
    allam_rewritten_texts = model.generate_text(prompts, concurrency_limit=len(prompts))

    for sample, allam_rewritten_text in zip(input_samples, allam_rewritten_texts):
        sample.append(allam_rewritten_text)
        rewrites_data.append(sample)

100%|██████████| 10/10 [02:16<00:00, 13.63s/it]


In [20]:
len(rewrites_data)

30

In [21]:
tgt_author_names, tgt_author_examples, input_author_names, inputs, allam_outputs = list(zip(*rewrites_data))

In [22]:
rewritten_df = pd.DataFrame({
    'target_author_name': tgt_author_names,
    'target_author_examples': tgt_author_examples,
    'input_author_name': input_author_names,
    'input': inputs,
    'rewrite': allam_outputs
})

In [23]:
predicted_classes = classify(rewritten_df['rewrite'].tolist())

In [24]:
len(predicted_classes)

30

In [25]:
predicted_authors = [idx_to_author_mapping[idx] for idx in predicted_classes]
rewritten_df['predicted_author_name'] = predicted_authors

In [26]:
rewritten_df.sample(n=5)

,target_author_name,target_author_examples,input_author_name,input,rewrite,predicted_author_name
24,نوال السعداوي,أذكر أنَّ صلاح أبو سيف أراد أن يُقدِّم إحدى رو...,سليم حسن,تدل الظواهر على أن «بيعنخي» قد تولى عرش ملك «م...,"استنادًا إلى المعلومات المتوفرة، يبدو أن ""...",سليم حسن
15,عباس محمود العقاد,قال قائل من سماسرة شوقي: ما ترى في رثائه لمصطف...,نقولا حداد,الحياة سفينة تقل الإنسان، وقد ألقتها الطبيعة ف...,الحياة سفينة تبحر في خضم العمران، وقد ألقت...,نقولا حداد
20,محمد لطفي جمعة,أما في الإسلام فقد اتخذ بعض علماء المشرقيات ال...,أبو حيان التوحيدي,قال أبو علي مسكويه، رحمه الله: إن الصدق والكذب...,قال أبو علي مسكويه، رحمه الله: الصدق والكذ...,أبو حيان التوحيدي
1,أبو حيان التوحيدي,وعُدْتُ ليلةً أخرى وقرأتُ عليه أشياء من هذا ال...,ثورنتون دبليو برجس,يَعْشَقُ كَلْبُ الصَّيْدِ باوزر الصَّيْدَ مِنْ...,في إحدى الليالي، قرأت على مسامع الشيخ جزءً...,أبو حيان التوحيدي
6,زكي نجيب محمود,لست أقيس قامتي إلى ذرة من «وِردِزْوِرْث» أو «ك...,سليم حسن,رأينا فيما سبق أن «بطليموس فيلومتور» كان منتصر...,"في سياق سابق، شهدنا انتصار ""بطليموس فيلومت...",سليم حسن


In [27]:
tgt_eq_pred_mask = rewritten_df['target_author_name']==rewritten_df['predicted_author_name']
print('EXACTLY TARGET AUTHOR', len(rewritten_df[tgt_eq_pred_mask]))       # 19
print('NOT EXACTLY TARGET AUTHOR', len(rewritten_df[~tgt_eq_pred_mask]))  # 11

EXACTLY TARGET AUTHOR 5
NOT EXACTLY TARGET AUTHOR 25


In [28]:
unmodified_input_tgt_author_probs = classify_and_get_prob(rewritten_df['input'].tolist(), rewritten_df['target_author_name'].tolist())
rewritten_input_tgt_author_probs = classify_and_get_prob(rewritten_df['rewrite'].tolist(), rewritten_df['target_author_name'].tolist())

In [29]:
unmodified_input_tgt_author_probs = pd.Series(unmodified_input_tgt_author_probs)
rewritten_input_tgt_author_probs = pd.Series(rewritten_input_tgt_author_probs)

In [30]:
(rewritten_input_tgt_author_probs > unmodified_input_tgt_author_probs).sum()

25

In [33]:
rewritten_df.to_csv('rewritten.csv', index=False)